In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
from tqdm import tqdm
import math, os, torch

# ==========================
# 🔧 CONFIGURATION
# ==========================

input_file = "/content/GPC OG only Bricks no Duplicates.xlsx"
output_file = f"GPC only Bricks Arabic Translated.xlsx"

model_name = "Helsinki-NLP/opus-mt-en-ar"
batch_size = 32
save_every = 500  # Save progress every 500 rows

# ==========================
# 🚀 MODEL INITIALIZATION
# ==========================
print("🔄 Loading model and tokenizer...")
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"✅ Model loaded on device: {device}")

# ==========================
# 📂 DATA LOADING
# ==========================
print(f"📖 Reading data from {input_file}...")
df = pd.read_excel(input_file)

# REMOVE slicing → translate full dataset
df = df.reset_index(drop=True)

# Resume progress if exists
if os.path.exists(output_file):
    translated_df = pd.read_excel(output_file)
    print(f"🔁 Resuming from existing progress: {output_file}")
else:
    translated_df = pd.DataFrame()

# ==========================
# 🧠 TRANSLATION FUNCTION
# ==========================
def translate_batch(texts):
    # Skip empty or NaN entries
    texts = [t if isinstance(t, str) and t.strip() else "" for t in texts]
    prefixed_texts = [">>ara<< " + t for t in texts]

    inputs = tokenizer(prefixed_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    translated = model.generate(**inputs, max_length=512)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# ==========================
# ⚙ TRANSLATION LOOP
# ==========================
print("🌍 Starting translation process...\n")

for col in df.columns:
    print(f"🈶 Translating column: {col}")
    texts = df[col].astype(str).tolist()
    translated_texts = []

    for i in tqdm(range(0, len(texts), batch_size), desc=f"Translating {col}"):
        batch = texts[i:i + batch_size]
        translated_batch = translate_batch(batch)
        translated_texts.extend(translated_batch)

        # Save progress every save_every rows
        if i % (save_every * batch_size) == 0 and i > 0:
            partial_df = pd.concat([df, translated_df], axis=1)
            partial_df.to_excel(output_file, index=False)

    # Add translated column to translated_df
    translated_df[col + "_ar"] = translated_texts

# ==========================
# 💾 FINAL SAVE
# ==========================
final_df = pd.concat([df, translated_df], axis=1)
final_df.to_excel(output_file, index=False)

print(f"\n✅ Translation complete for !")
print(f"💾 Saved to: {output_file}")
